## Simuation of the effects of missing data in the SVDquartets analysis
In order to explore the effect of missing data on the ability to recover the correct topology in SVDquartets a number of ddrad datasets will be simulated before various amounts of data will be convereted to 'N's, and then SVDquartets will be run.

Steps
* Make directory for new simulation
    * `mkdir l<length>L<nloci>`
    * `cd l<length>L<nloci>`
* Use *simrrls* (Eaton) to construct ddrad datasets
    * `simrrls -o l<length>L<nloci> -f ddrad -l <length> -L <nLoci>`
    * Loci lengths will range from 100 too 1000 in steps of 100
    * Number of loci will range from 100 to 600 in steps of 100
* Data will be run through pyRAD (Eaton) using default parameters
    * `pyrad -n`
    * Rename barcodes file from `./l<length>L<nloci>_barcodes.txt` to `./l<length>L<nloci>.barcodes`
    * Edit lins in params file
        * (6) Restriction overhand from `TGCAG` to `CTGCAG, CCGG`
        * (11) Datatype from `rad` to `ddrad`
        * (14) Prefix form `c88d6m4p3` to `l<length>L<nloci>.c88d6m4p3`
    * `pyrad -p params.txt -s 1234567`
* Data will be analyzed via SVDquartets in PAUP
    * Write PUAP block to nexus file by appending `~/simrrls/SVD_paupblock.txt`
        * `cat SVD_paupblock.txt >> ./outfiles/*.nex`
    * `paup4 ./outfiles/l<length>L<nloci>.nex
    

### 0. Load modules

In [1]:
try: import sys
except ImportError:
    print "\n\tError: sys is not installed/loaded"
    sys.exit()
try: import os
except ImportError:
    print "\n\tError: os is not installed/loaded"
    sys.exit()
try: from optparse import OptionParser
except ImportError:
    print "\n\tError: OptionParser (optparse) is not installed/loaded"
    sys.exit()
try: import shutil
except ImportError:
    print "\n\tError: shutil is not installed/loaded"
    sys.exit()
try: from tqdm import tqdm, tqdm_notebook, tnrange
except ImportError:
    print "\n\tError: tqdm is not installed/loaded."
    print "\n\tTo install, try 'pip install tqdm' or 'conda install -c conda-forge tqdm'"
    sys.exit()
import subprocess
from pprint import pprint
import numpy as np
import pandas as pd
import numpy.random as rand
from Bio import AlignIO, Alphabet
import re
import dendropy
import csv

In [3]:
def listabs(directory):
    return [os.path.join(directory, filename) for filename in os.listdir(directory)]

class Vividict(dict):
    def __missing__(self, key):
        value = self[key] = type(self)() # retain local pointer to value
        return value                     # faster to return than dict lookup

In [2]:
os.chdir('/Users/IanGilman/Desktop/TempSims/L100/l100L100SluNh/')

In [17]:
reps=10
rfpath = './l100L100SluNh.RF.csv'
if os.path.exists(rfpath):
    with open(rfpath, 'r') as r:
        if not len(r.readlines())==reps+1:
            print('Found incomplete RF table. Deleting and recalculating.\n')
            os.remove(rfpath)
        else: 
            print('Found RF distance matrix for simulation %s, move/remove to recalculate. Continuing...\n') % ('BLAH')
print('Hello')

Found incomplete RF table. Deleting and recalculating.

Hello


### 1. Simulate datasets

In [4]:
print os.getcwd()
simrrls_dir = '/Users/IanGilman/simrrls/'
os.chdir(simrrls_dir)
print os.getcwd()
newsimsdir = '/Users/IanGilman/simrrls/newsims/'
os.chdir(newsimsdir)
print os.getcwd()

/Users/IanGilman/pythonscripts
/Users/IanGilman/simrrls
/Users/IanGilman/simrrls/newsims


nuc Spinach/Silene 15.8-31.5 
    
   Monocot/dicot 5.8- 8.1

In [5]:
def multiply_strints(mystring, by):
    return re.sub(
        re.compile("\d+"), 
        lambda matchobj: str(int(matchobj.group(0))*by), mystring)

In [6]:
MedBalTree = '((((((A:2,B:2):2,C:4):4,D:8):4,(((E:2,F:2):2,G:4):4,H:8):4):4,(((I:2,J:2):2,K:4):4,L:8):8):16);'
MedUnbTree = '((((((((((((A:1,B:1):1,C:2):1,D:3):1,E:4):1,F:5):1,G:6):1,H:7):1,I:8):1,J:9):1,K:10):1,L:11):12);'
ShortBalTree = multiply_strints(MedBalTree, 0.5)
LongBalTree = multiply_strints(MedBalTree, 2.0)
ShortUnbTree = multiply_strints(MedUnbTree, 0.5)
LongUnbTree = multiply_strints(MedUnbTree, 2.0)

In [7]:
treeshapes = Vividict()
treeshapes['ShortBalTree']['tree'] = ShortBalTree
treeshapes['MedBalTree']['tree'] = MedBalTree
treeshapes['LongBalTree']['tree'] = LongBalTree
treeshapes['ShortUnbTree']['tree'] = ShortUnbTree
treeshapes['MedUnbTree']['tree'] = MedUnbTree
treeshapes['LongUnbTree']['tree'] =LongUnbTree

In [8]:
for key in treeshapes.iterkeys():
    path = os.path.join(simrrls_dir, key)
    treeshapes[key]['path'] = str(path)
    with open(path, 'w+') as f:
        f.write(treeshapes[key]['tree'])

In [9]:
treeshapes['sb'] = treeshapes.pop('ShortBalTree')
treeshapes['mb'] = treeshapes.pop('MedBalTree')
treeshapes['lb'] = treeshapes.pop('LongBalTree')
treeshapes['su'] = treeshapes.pop('ShortUnbTree')
treeshapes['mu'] = treeshapes.pop('MedUnbTree')
treeshapes['lu'] = treeshapes.pop('LongUnbTree')

In [10]:
nloci = [100, 500, 1000, 5000, 10000]
lengths = [100, 200, 300, 400, 500, 600]
Nes = {'l':100, 'm': 1000, 'h':10000}

combos = []
for number in nloci:
    for size in lengths:
        for shape in treeshapes.keys():
            for Ne in Nes.keys():
                combos.append([size, number, shape, Ne])
print(len(combos))
print combos

540
[[100, 100, 'mb', 'h'], [100, 100, 'mb', 'm'], [100, 100, 'mb', 'l'], [100, 100, 'su', 'h'], [100, 100, 'su', 'm'], [100, 100, 'su', 'l'], [100, 100, 'lb', 'h'], [100, 100, 'lb', 'm'], [100, 100, 'lb', 'l'], [100, 100, 'mu', 'h'], [100, 100, 'mu', 'm'], [100, 100, 'mu', 'l'], [100, 100, 'lu', 'h'], [100, 100, 'lu', 'm'], [100, 100, 'lu', 'l'], [100, 100, 'sb', 'h'], [100, 100, 'sb', 'm'], [100, 100, 'sb', 'l'], [200, 100, 'mb', 'h'], [200, 100, 'mb', 'm'], [200, 100, 'mb', 'l'], [200, 100, 'su', 'h'], [200, 100, 'su', 'm'], [200, 100, 'su', 'l'], [200, 100, 'lb', 'h'], [200, 100, 'lb', 'm'], [200, 100, 'lb', 'l'], [200, 100, 'mu', 'h'], [200, 100, 'mu', 'm'], [200, 100, 'mu', 'l'], [200, 100, 'lu', 'h'], [200, 100, 'lu', 'm'], [200, 100, 'lu', 'l'], [200, 100, 'sb', 'h'], [200, 100, 'sb', 'm'], [200, 100, 'sb', 'l'], [300, 100, 'mb', 'h'], [300, 100, 'mb', 'm'], [300, 100, 'mb', 'l'], [300, 100, 'su', 'h'], [300, 100, 'su', 'm'], [300, 100, 'su', 'l'], [300, 100, 'lb', 'h'], [300, 

### 2. Run simRRLs and pyRAD

In [310]:
def run_simrrls(size, nloci, shape, ne, simname):
    tree = treeshapes[shape]['path']
    simrrls_call = 'simrrls -o %s -f ddrad -l %d -L %d -N %s' % (simname, size, nloci, Nes[ne])
    if treeshapes[shape]:
        simrrls_call = 'simrrls -o %s -f ddrad -l %d -L %d -t %s -N %s' % (simname, size, nloci, tree, Nes[ne])
    print('Running simrrls...\n%s\n') % (simrrls_call)
    subprocess.call(simrrls_call, shell=True)
    print('Dataset %s finished.\n') % (simname)

In [313]:
def run_pyrad(simname, pyrad = 'python ~/pyrad/pyrad/pyRAD.py '):
    
    # Create new params file
    new_params_call = pyrad+'-n'
    subprocess.call(new_params_call, shell=True)
    
    # Rename barcodes file so it's recognized by default by pyRAD
    oldbcfile = simname+'_barcodes.txt'
    newbcfile = simname+'.barcodes'
    if not os.path.exists(newbcfile):
        shutil.move(oldbcfile, newbcfile)
    
    # Read in params file
    with open('params.txt', 'r') as p:
        paramsdata =  p.readlines()
    
    # Edit lines
    newlines = []
    for line in paramsdata:
        if 'Restriction overhang' in line:
            line = 'CTGCAG, CCGG                     ## 6. Restriction overhang (e.g., C|TGCAG -> TGCAG)     (s1,s2)\n'
        if 'Prefix name' in line:
            line = '%s.c88d6m4p3                 ## 14. Prefix name for final output (no spaces)         (s7)\n' % (simname)
        if 'output formats' in line:
            line = '*                       ## 30.opt.: output formats: p,n,a,s,v,u,t,m,k,g,* (see docs) (s7)\n'
        if 'Datatype' in line:
            line = 'ddrad                       ## 11. Datatype: rad,gbs,pairgbs,pairddrad,(others:see docs)(all)\n'
        newlines.append(line)
    
    # Write new lines
    with open('params.txt', 'w') as p:
        p.writelines(newlines)
    
    # Run pyRad
    pyrad_call = pyrad+'-p params.txt -s 1234567'
    print(pyrad_call)
    subprocess.call(pyrad_call, shell=True)

### 3. Create missing data replicates
Idea: could we create replicated by simulating missing data multiple times instead of simRRLs and pyRAD?

In [118]:
# flatten = lambda l: [item for sublist in l for item in sublist]

In [314]:
def phylip_to_df(simname):
    
    # Read phylip data
    phylip = './outfiles/%s.c88d6m4p3.phy' % (simname)
    with open(phylip, 'r') as p:
        phylipdata = p.readlines()
    
    # Split all sites 
    splitdata = []
    for line in phylipdata[1:]:
        splitdata.append(line.split())
    for i, line in enumerate(splitdata):
        splitdata[i][1] = list(splitdata[i][1])    
    flatdata = []
    for line in splitdata:
        flatdata.append([line[0]]+line[1])
    
    # Write to dataframe
    datadf = pd.DataFrame(flatdata)
    datadf.set_index(keys=[0], inplace=True)
    
    return datadf

In [350]:
def write_missing_data(fulldf, percent, simname):
    
    # Create string for naming files
    p = '.p'+str(percent).zfill(2)
    
    # Delete missing data from previous replicate if it exists
    if os.path.exists(simname+p+'.nex'):
        os.remove(simname+p+'.nex')
    if os.path.exists(simname+p+'.phy'):
        os.remove(simname+p+'.phy')
    
    # Get number of entries
    entries = fulldf.shape[0]*fulldf.shape[1]
    
    # Substitute rand percent with 'N'
    if percent==0: missingdf=fulldf
    else: missingdf = fulldf.where(np.random.uniform(size=fulldf.shape) > float(percent/100.), 'N')
    
    # Convert dataframe to sequence strings
    missinglines =[]
    meta = ('%d %d\n') % (fulldf.shape[0], fulldf.shape[1])
    missinglines.append(meta)
    for i in xrange(len(missingdf)):
        missinglines.append('      '.join([missingdf.index[i],''.join(missingdf.ix[i])])+'\n')
    
    # Write strings to phylip file
    with open(simname+p+'.phy', 'w+') as m:
        m.writelines(missinglines)
        #print('Wrote %d missing data phylip file to %s') % (percent, simname+p+'.phy')
    # Convert phlyip to nexus for PAUP
    alignment = AlignIO.read(open(simname+p+'.phy'), "phylip", alphabet=Alphabet.generic_dna)
    with open(simname+p+'.nex', "w+") as n:
        n.write(alignment.format("nexus"))
        #print('Wrote %d missing data nexus file to %s') % (percent, simname+p+'.nex')
        
    return simname+p+'.nex'

### 4. Run SVDquartets

In [351]:
def write_svdq_block(nexus, percent, blockpath='/Users/IanGilman/simrrls/SVD_paupblock.txt'):
    
    # Read default PAUP block for funning SVDquartets
    with open(blockpath, 'r') as b:
        defaultblock = b.readlines()
        
    # Edit block lines
    newblock = ['\n']
    for line in defaultblock:
        if 'paup.log' in line:
            line = '\tlog file = p%s.paup.log start replace;\n' % (str(percent).zfill(3))
        if 'SVDquartets' in line:
            line = "\tSVDquartets evalQuartets=all;\n"
        if 'SaveTrees' in line:
            line = '\tSaveTrees file=./p%s.tre;\n' % (str(percent).zfill(3))
        newblock.append(line)

    # Append block to nexus file
    with open(nexus, 'a') as n:
        n.writelines(newblock)

In [394]:
print(os.getcwd())

## Loop over all combinations
for i, combo in tqdm_notebook(enumerate(combos[:10]), desc='Simulation'):
    
    ## For convenience
    size = combo[0]
    number = combo[1]
    shape = combo[2]
    ne = combo[3]
    
    # Create tree object from true topology
    truetree = dendropy.Tree.get_from_string(treeshapes[shape]['tree'], schema='newick')
    
    # Create dict to fill of tree distances
    percents = ['p'+str(rate).zfill(3) for rate in range(0,95, 10)]
    treedist = {}
    for key in percents:
        treedist[key] = []

    ## Create simulation name
    print('\nSimulation %d\nN loci:%d\nLocus length: %d\nTree shape: %s\nNe: %s\n') % (i+1, number, size, shape, ne)
    simname = 'l%sL%sS%sN%s' % (size, number, shape, ne)
    print('Simulation name: %s') % (simname)
    
    ## Create/move to simulation directory
    cursimdir = os.path.join(newsimsdir, simname)
    try: 
        os.mkdir(cursimdir)
        print('Created directory %s') % (cursimdir)
    except OSError: print('Directory %s exists') % (cursimdir)
    os.chdir(cursimdir)
    print('Current working directory: %s\n') % (os.getcwd())
    
    ## Run simulation and pyRAD if not already done
    if not os.path.exists(simname+'_R1_.fastq.gz'):
        run_simrrls(size=size, nloci=number, shape=shape, ne=ne, simname=simname)
    else: print('Dataset %s already simulated. Move/remove it to rerun. Continuing to pyRAD...\n') % (simname)
    run_pyrad(simname)
    print('... pyRAD finished.\n')
    
    ## Want to create and analyze 10 replicates of missing data
    ## Create missing data datasets
    print('Creating missing data datsets...\n')
    # Read phylip file to datframe 
    fulldata = phylip_to_df(simname=simname)
    
    print('Running SVDquartets and calculating RF distances...\n')
    for i in tqdm_notebook(range(10), desc='Missing data replicates', leave = False):
        # Write/replace nexus files and append PAUP blocks
        for mp in tqdm_notebook(range(0, 95, 10), desc='Percent missing', leave=False):
            nexusfile = write_missing_data(fulldf=fulldata, percent=mp, simname=simname)
            write_svdq_block(nexus=nexusfile, percent=mp)

        nexusfiles = [filename for filename in os.listdir(os.getcwd()) if filename.endswith('.nex')]
        for nexfile in tqdm_notebook(nexusfiles, desc='SVDquartets', leave=False):
            mp = nexfile.split('.')[1]
            run_svdq_call = 'paup4 -n %s' % (nexfile)
            subprocess.call(run_svdq_call, shell=True)

        treefiles = [filename for filename in os.listdir(os.getcwd()) if filename.endswith('.tre')]
        for tree in treefiles:
            key = tree.split('.')[0]
            t = dendropy.Tree.get_from_path(tree, schema='nexus')
            t.symmetric_difference(truetree)
            try: treedist[key].append(truetree.symmetric_difference(t))
            except KeyError: continue
    
    pprint(treedist)
    pd.DataFrame.from_dict(treedist).to_csv(path_or_buf=simname+'.RF.csv')
    
    print('\nFinished. RF distances written to %s\n') % (simname+'.RF.csv')

/Users/IanGilman/.Trash/l100L100SlbNh 1.39.31 PM

Simulation 1
N loci:100
Locus length: 100
Tree shape: mb
Ne: h

Simulation name: l100L100SmbNh
Created directory /Users/IanGilman/simrrls/newsims/l100L100SmbNh
Current working directory: /Users/IanGilman/simrrls/newsims/l100L100SmbNh

Running simrrls...
simrrls -o l100L100SmbNh -f ddrad -l 100 -L 100 -t /Users/IanGilman/simrrls/MedBalTree -N 900000.0

Dataset l100L100SmbNh finished.

python ~/pyrad/pyrad/pyRAD.py -p params.txt -s 1234567
... pyRAD finished.

Creating missing data datsets...

Running SVDquartets and calculating RF distances...

{'p000': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'p010': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'p020': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'p030': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'p040': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'p050': [0, 0, 0, 0, 0, 0, 0, 0, 2, 0],
 'p060': [0, 0, 6, 0, 0, 2, 0, 0, 2, 0],
 'p070': [0, 0, 0, 2, 0, 0, 0, 2, 2, 4],
 'p080': [6, 6, 8, 10, 8, 12, 16, 10, 8, 16],
 'p090': [18, 16, 18, 18, 

KeyboardInterrupt: 

## 5. Run GARLI
For all loci from all runs we will run GARLI to create input for ASTRAL

* Read in GARLI.conf example from `~/Garli/example/basic/garli.conf`
* Make list of all directories
* For each directory, `cd` and make list of nexus files
* For each nexus file create a new garli.conf file
    * 
* Run GARLI for 5 trees? This is 60 (datasets) x 16 (missing data rates) x 5 (trees)= 4800 trees to estimate

## 6. Copy all files to directory for ASTRAL analysis

In [5]:
1e5

100000.0